In [76]:
import numpy as np
import pandas as pd
import re
import cupy as cp
from tqdm import tqdm
import spacy
from spacy.tokens import Doc
from functools import partial
import sys
sys.path.append('c:\\python312\\lib\\site-packages')
# Load the data
df = pd.read_csv("data/bbc-text.csv")
def create_vocabulary(complete_text):
    text = re.sub(r'[^\w\s]', ' ', complete_text).split()
    vocabulary = list(set(text)) 
    vocabulary = {word: idx for idx, word in enumerate(vocabulary)} 
    print("Vocabulary size: ", len(vocabulary))
    return vocabulary

complete_text = ' '.join(df["text"].tolist()) 
vocabulary= create_vocabulary(complete_text)
vocabulary

Vocabulary size:  29457


{'intertwined': 0,
 'perpetuating': 1,
 'dolls': 2,
 'compton': 3,
 'purporting': 4,
 'ilw': 5,
 'rice': 6,
 'feted': 7,
 'mccurry': 8,
 'qualification': 9,
 'restructuring': 10,
 'redesigned': 11,
 'viktor': 12,
 '56bn': 13,
 'elene': 14,
 'pointed': 15,
 '15m': 16,
 'fork': 17,
 'treasure': 18,
 'icstis': 19,
 'united': 20,
 'walshe': 21,
 'tribunal': 22,
 'chesney': 23,
 'creations': 24,
 'freak': 25,
 'achilles': 26,
 'tutsis': 27,
 'landing': 28,
 'woman': 29,
 'consulted': 30,
 'maximise': 31,
 'malone': 32,
 'apocalypse': 33,
 'commoditising': 34,
 'bouira': 35,
 'musicians': 36,
 'cockerill': 37,
 'las': 38,
 'speechless': 39,
 'ferdinand': 40,
 '110m': 41,
 '76m': 42,
 'fenton': 43,
 'husbands': 44,
 'rugged': 45,
 'barclay': 46,
 'handbook': 47,
 'rare': 48,
 'chains': 49,
 'dame': 50,
 'noting': 51,
 'basf': 52,
 'honor': 53,
 'spectacularly': 54,
 'preston': 55,
 'alzheimer': 56,
 'foment': 57,
 'dying': 58,
 'sectors': 59,
 'emerged': 60,
 'server': 61,
 'offer': 62,
 'duf

In [77]:
def input_one_hot_vectors(words,vocabulary):
    
    inputs=np.zeros((len(words),len(vocabulary)),int)
     
    for i in range(len(words)): 
        inputs[i][vocabulary[words[i]]]=1
    
    return inputs 

word=cp.array(input_one_hot_vectors(['intertwined'],vocabulary))
word.shape,words_len_embedding_layer.shape,words_len_embedding_bias.shape

((1, 29457), (29457, 100), (100,))

In [78]:
import pickle
with open('data/words_len_embedding_layer.pkl', 'rb') as f:
    words_len_embedding_layer = pickle.load(f)

with open('data/words_len_embedding_bias.pkl', 'rb') as f:
    words_len_embedding_bias = pickle.load(f)

# with open('data/outlayer_maps_vocab_average.pkl', 'rb') as f:
#     outlayer_maps_vocab_average = pickle.load(f)

# with open('data/out_bias_maps_vocab_average.pkl', 'rb') as f:
#     out_bias_maps_vocab_average = pickle.load(f)

# with open('data/outlayer_maps_vocab_concat.pkl', 'rb') as f:
#     outlayer_maps_vocab_concat = pickle.load(f)

# with open('data/out_bias_maps_vocab_concat.pkl', 'rb') as f:
#     out_bias_maps_vocab_concat = pickle.load(f)

In [134]:
import numpy as np
import cupy as cp

words = []  # List to store words
embeddings_list = []  # List to store word embeddings
vocabulary_embedded=dict()
def produce_embedding(word,words_len_embedding_layer,words_len_embedding_bias):
    
    # Create one-hot vector for the word using CuPy
    word_one_hot = cp.array(input_one_hot_vectors([word], vocabulary)) 
    word_vector = word_one_hot @ words_len_embedding_layer + words_len_embedding_bias
    vocabulary_embedded[word]=word_vector
    return word_vector
    

# Loop over your vocabulary
for word in vocabulary.keys():
    words.append(word)  # Append word to the list 
    # Compute the word vector using matrix multiplication (assuming `words_len_embedding_layer` and `words_len_embedding_bias` are CuPy arrays)
    word_vector = produce_embedding(word,words_len_embedding_layer,words_len_embedding_bias)
    
    # Append the computed word vector to the list
    embeddings_list.append(word_vector.get())  # Convert to NumPy array before appending

# After the loop, convert the list of embeddings to a NumPy array
# words_embeddings=np.array([x[0]  for x in embeddings_list] ) 

# # Save the embeddings to a TSV file
# np.savetxt('data/learned_embeddings.tsv', words_embeddings, delimiter='\t')

# # Save the words to a metadata TSV file
# with open('data/metadata.tsv', 'w') as f:
#     for word in words:
#         f.write(f"{word}\n")


In [106]:
def cosine_similarity(A, B):
    # Flatten the arrays to treat them as 1D vectors
    A = A.flatten()
    B = B.flatten()
    
    dot_product = np.dot(A, B)
    magnitude_A = np.linalg.norm(A)
    magnitude_B = np.linalg.norm(B)
    
    return dot_product / (magnitude_A * magnitude_B)

In [126]:
A=produce_embedding("king",words_len_embedding_layer,words_len_embedding_bias)

In [127]:
B=produce_embedding("man",words_len_embedding_layer,words_len_embedding_bias)

In [128]:
C=produce_embedding("woman",words_len_embedding_layer,words_len_embedding_bias)

In [129]:
D=produce_embedding("queen",words_len_embedding_layer,words_len_embedding_bias)

In [130]:
w=A-B+C

In [133]:
cosine_similarity(A, D)

array(0.98976086)

In [111]:
A.shape,B.shape,C.shape

((1, 100), (1, 100), (1, 100))

In [138]:
df["text"]

0       tv future in the hands of viewers with home th...
1       worldcom boss  left books alone  former worldc...
2       tigers wary of farrell  gamble  leicester say ...
3       yeading face newcastle in fa cup premiership s...
4       ocean s twelve raids box office ocean s twelve...
                              ...                        
2220    cars pull down us retail figures us retail sal...
2221    kilroy unveils immigration policy ex-chatshow ...
2222    rem announce new glasgow concert us band rem h...
2223    how political squabbles snowball it s become c...
2224    souness delight at euro progress boss graeme s...
Name: text, Length: 2225, dtype: object

In [135]:
vocabulary_embedded["queen"]

array([[ 1.09329447, 21.64923463,  1.8612367 ,  0.27472562,  1.52338093,
         0.59527578,  1.80747309,  0.55690797,  1.60238616,  1.90442172,
         0.6202352 ,  1.8844113 ,  1.64926506,  1.23952358,  1.30180696,
         1.18868737,  1.49898034,  0.529768  ,  0.69477201,  1.07205463,
         0.55677799, -0.22342586,  1.43369831,  1.1980755 ,  0.68340263,
         1.38998833, 11.5009156 ,  0.90735146,  0.44628396,  0.98617331,
         1.58640365,  1.53239808,  1.58673176,  0.50128468,  0.99522132,
         1.07413335,  1.82815961,  1.40472235,  1.16222644,  1.07584088,
         1.03151686,  1.78626986,  0.86363062,  1.10198276,  1.33244743,
         1.06649008,  1.15666435,  0.95016706,  1.98556134,  0.31493584,
         1.83630429,  1.05954446,  0.90359978,  0.85184906,  1.36591391,
         0.65626496,  0.69237014,  1.11592008,  1.05276219,  1.76675102,
         1.42630853,  0.35395692,  1.86676033,  0.39550055,  0.54199053,
         0.64974436,  0.08705415,  0.51532393,  2.1

In [144]:
complete_inp=[]
for i in range(len(df)):
    inp=[]
    print(i)
    for j in re.sub(r'[^\w\s]', ' ', df["text"].iloc[i]).split():
        inp.append(vocabulary_embedded[j])
    complete_inp.append(inp)
        
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [146]:
np.array(complete_inp).shape

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [148]:
np.array([x[0]  for x in complete_inp] ) 

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.

In [152]:
len(re.sub(r'[^\w\s]', ' ', df["text"].iloc[0]).split())

750

In [150]:
len(complete_inp[0])

750

In [153]:
np.array(complete_inp[0]).shape

TypeError: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.